<a href="https://colab.research.google.com/github/ducleins/eurusd_2026/blob/main/02_Notebook/01_Download_All_Raw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# =========================================================
# CELL 1: SETUP, CÀI ĐẶT VÀ THIẾT LẬP API KEY
# =========================================================

# Cài đặt thư viện cần thiết (Chỉ cần chạy nếu cần)
!pip install pandas_datareader fredapi yfinance

import pandas as pd
import numpy as np
import os
from fredapi import Fred
import yfinance as yf
from pandas_datareader import data as pdr
from datetime import datetime
from google.colab import drive # Import thư viện Drive

# --- 1. KẾT NỐI GOOGLE DRIVE (BẮT BUỘC CHẠY DÒNG NÀY ĐẦU TIÊN) ---
drive.mount('/content/drive')

# --- 2. ĐỊNH NGHĨA CÁC ĐƯỜNG DẪN CHÍNH ---
BASE_PATH = '/content/drive/MyDrive/EURUSD_2026'
RAW_PATH = os.path.join(BASE_PATH, '01_Raw')
os.makedirs(RAW_PATH, exist_ok=True)

# --- 3. THIẾT LẬP FRED API KEY ---
FRED_API_KEY = '424b31a7fb36e1da7094435bdffa2a37'
fred = Fred(api_key=FRED_API_KEY)

# Ngày bắt đầu tải dữ liệu (để đồng bộ các series)
START_DATE = datetime(2018, 1, 1)

print("✅ Thiết lập môi trường và API thành công. Sẵn sàng tải dữ liệu.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Thiết lập môi trường và API thành công. Sẵn sàng tải dữ liệu.


In [2]:
# =========================================================
# CELL 2: TẢI DỮ LIỆU FRED VĨ MÔ
# =========================================================

def download_fred_data():
    """Tải và lưu trữ tất cả các series FRED vào thư mục 01_Raw/fred."""

    FRED_DIR = os.path.join(RAW_PATH, 'fred')
    os.makedirs(FRED_DIR, exist_ok=True)

    # Danh sách các series FRED cần tải (Tên file: FRED ID)
    fred_series = {
        'FRED_WALCL.csv': 'WALCL',
        'FRED_DGS10.csv': 'DGS10',
        'FRED_DGS2.csv': 'DGS2',
        'FRED_VIXCLS.csv': 'VIXCLS',
        'FRED_RRPONTSYD.csv': 'RRPONTSYD',
        'FRED_TGA.csv': 'WTREGEN',
        'FRED_EUR3M.csv': 'IR3TIB01EZM156N',  # Đã thay ID cũ 'EURIBOR3MTD' bằng ID mới
        'FRED_SOFR.csv': 'SOFR90DAYAVG',      # Đã thay ID cũ 'SOFR3MS' bằng ID mới
        'FRED_DTWEXBGS.csv': 'DTWEXBGS'
    }

    print("\n--- BẮT ĐẦU TẢI DỮ LIỆU TỪ FRED ---")

    for filename, series_id in fred_series.items():
        try:
            # Tải dữ liệu từ ngày START_DATE
            df = fred.get_series(series_id, observation_start=START_DATE)
            if df is not None and not df.empty:
                df.name = filename.replace('FRED_', '').replace('.csv', '')
                df.to_csv(os.path.join(FRED_DIR, filename), header=True)
                print(f"✅ Tải thành công: {filename}")
            else:
                print(f"⚠️ Không tìm thấy dữ liệu cho: {filename} ({series_id})")
        except Exception as e:
            print(f"❌ Lỗi tải {filename} ({series_id}): {e}")

download_fred_data()


--- BẮT ĐẦU TẢI DỮ LIỆU TỪ FRED ---
✅ Tải thành công: FRED_WALCL.csv
✅ Tải thành công: FRED_DGS10.csv
✅ Tải thành công: FRED_DGS2.csv
✅ Tải thành công: FRED_VIXCLS.csv
✅ Tải thành công: FRED_RRPONTSYD.csv
✅ Tải thành công: FRED_TGA.csv
✅ Tải thành công: FRED_EUR3M.csv
✅ Tải thành công: FRED_SOFR.csv
✅ Tải thành công: FRED_DTWEXBGS.csv


In [6]:
# =========================================================
# CELL 3: TẢI DỮ LIỆU YFINANCE
# =========================================================

def download_yfinance_data():
    """Tải tỷ giá EURUSD và lưu trữ vào 01_Raw/yfinance."""

    YFINANCE_DIR = os.path.join(RAW_PATH, 'yfinance')
    os.makedirs(YFINANCE_DIR, exist_ok=True)

    TICKER = 'EURUSD=X'
    FILE_NAME = 'yfinance_EURUSD_daily.csv'

    print("\n--- BẮT ĐẦU TẢI DỮ LIỆU YFINANCE ---")

    try:
        # Tải dữ liệu OHLCV (Open, High, Low, Close, Volume)
        df_euraud = yf.download(TICKER, start=START_DATE, progress=False)

        if not df_euraud.empty:
            df_euraud.to_csv(os.path.join(YFINANCE_DIR, FILE_NAME))
            print(f"✅ Tải thành công: {FILE_NAME}")
        else:
            print(f"⚠️ Không tìm thấy dữ liệu cho mã {TICKER}")

    except Exception as e:
        print(f"❌ Lỗi tải YFinance ({TICKER}): {e}")

download_yfinance_data()


--- BẮT ĐẦU TẢI DỮ LIỆU YFINANCE ---


/tmp/ipython-input-231039860.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_euraud = yf.download(TICKER, start=START_DATE, progress=False)


✅ Tải thành công: yfinance_EURUSD_daily.csv


In [7]:
# =========================================================
# CELL 4: CHUẨN HÓA DỮ LIỆU YFINANCE (LẤY DATE VÀ CLOSE)
# =========================================================

# Giả định RAW_PATH và START_DATE đã được định nghĩa ở các CELL trên

# --- A. ĐỊNH NGHĨA ĐƯỜNG DẪN ---
YFINANCE_DIR = os.path.join(RAW_PATH, 'yfinance')
FILTERED_PATH = os.path.join(RAW_PATH, 'filtered')
os.makedirs(FILTERED_PATH, exist_ok=True)

YFINANCE_FILE_RAW = os.path.join(YFINANCE_DIR, 'yfinance_EURUSD_daily.csv')
YFINANCE_FILE_CLEAN = os.path.join(FILTERED_PATH, 'yfinance_EURUSD_clean.csv')

if not os.path.exists(YFINANCE_FILE_RAW):
    print(f"❌ Không tìm thấy file YFinance thô tại: {YFINANCE_FILE_RAW}. Đã bỏ qua bước chuẩn hóa.")
else:
    # --- B. ĐỌC VÀ CHUẨN HÓA ---
    try:
        # Đọc file YFinance thô (có MultiIndex Header: Price/Ticker và Date)
        df_yf = pd.read_csv(
            YFINANCE_FILE_RAW,
            index_col=0,            # Cột 0 là cột Date
            parse_dates=True,
            header=[0, 1]           # Header là MultiIndex (2 hàng)
        )

        # 1. Chuẩn hóa: Chỉ chọn cột 'Close' của 'EURUSD=X' và đổi tên
        # Kết quả là một Series với Date là Index và giá trị là Close
        df_close = df_yf['Close']['EURUSD=X'].rename('EURUSD_Close')

        # 2. LƯU VÀO THƯ MỤC filtered
        # Xuất ra file CSV mới chỉ gồm cột Date và EURUSD_Close
        df_close.to_csv(YFINANCE_FILE_CLEAN, header=True)

        print(f"✅ Chuẩn hóa YFinance thành công. Đã lưu file sạch tại: {YFINANCE_FILE_CLEAN}")

    except Exception as e:
        print(f"❌ Lỗi chuẩn hóa YFinance: {e}")

✅ Chuẩn hóa YFinance thành công. Đã lưu file sạch tại: /content/drive/MyDrive/EURUSD_2026/01_Raw/filtered/yfinance_EURUSD_clean.csv


In [ ]:
# Lệnh 1: Kiểm tra xem ổ đĩa đã được gắn (mount) chưa
!ls /content/drive/MyDrive

# Lệnh 2: Xem nội dung chính xác của thư mục FRED
!ls /content/drive/MyDrive/EURUSD_2026/01_Raw/fred